In [4]:
#!pip install cassandra-driver
import cassandra

In [19]:
from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
    print('session started')
except Exception as e:
    print(e)

session started


In [28]:
#Create keyspace to work in 
try: 
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS james_keyspace 
    WITH REPLICATION = 
    {'class': 'SimpleStrategy', 'replication_factor' : 1}
    """)

except Exception as e:
    print(e)

In [29]:
try:
    session.set_keyspace('james_keyspace')
except Exception as e:
    print(e)

In [34]:
drop1  = "DROP TABLE IF EXISTS music_library"
drop2  = "DROP TABLE IF EXISTS album_library"
drop3 = "DROP TABLE IF EXISTS artist_library"

try: 
    session.execute(drop1)
    session.execute(drop2)
    session.execute(drop3)
except Exception as e: 
    print(e)

In [43]:
#Create table
def create_table(session, query, table_name):
    """  
    Executes a table creation query and handles excpetions. 

    :param session: Cassandra session object
    :param query: CQL query to create table|
    :param table_name: Names of table to be created
    """
    try: 
        session.execute(query)
        print(f"Table '{table_name}' created sucessfully")
    except Exception as e: 
        print(f"Error creating '{table_name}': {e}")

create_music_library = """
    CREATE TABLE IF NOT EXISTS music_library(
        year int,
        artist_name text, 
        album_name text, 
        PRIMARY KEY (year, artist_name)
        );
"""

create_album_library = """
    CREATE TABLE IF NOT EXISTS album_library(
        album_name text,
        artist_name text,
        year int, 
        PRIMARY KEY (album_name, artist_name)
    );
"""

create_artist_library = """
    CREATE TABLE IF NOT EXISTS artist_library(
        artist_name text,
        year int,
        album_name text, 
        
        PRIMARY KEY (artist_name, year)

    
    );
"""

create_table(session, create_music_library, 'music_library')
create_table(session, create_album_library, 'album_library')
create_table(session, create_artist_library, 'artist_library')


        

Table 'music_library' created sucessfully
Table 'album_library' created sucessfully
Table 'artist_library' created sucessfully


In [44]:
# Define a helper function for executing queries
def execute_insert(session, query, row):
    """
    Executes the given insert query for the provided row data.
    
    :param session: Cassandra session object
    :param query: Query to execute
    :param row: A dictionary containing the data to insert
    """
    try:
        session.execute(query, row)
        print(f"Inserted: {row}")
    except Exception as e:
        print(f"Error inserting {row}: {e}")


# Insert queries for the three tables
query_music = "INSERT INTO music_library (year, artist_name, album_name) VALUES (%s, %s, %s)"
query_artist = "INSERT INTO artist_library (artist_name, year, album_name) VALUES (%s, %s, %s)"
query_album = "INSERT INTO album_library (album_name, artist_name, year) VALUES (%s, %s, %s)"

# Data to insert
data = [
    {"year": 1970, "artist_name": "The Beatles", "album_name": "Let it Be"},
    {"year": 1965, "artist_name": "The Beatles", "album_name": "Rubber Soul"},
    {"year": 1965, "artist_name": "The Who", "album_name": "My Generation"},
    {"year": 1966, "artist_name": "The Monkees", "album_name": "The Monkees"},
    {"year": 1970, "artist_name": "The Carpenters", "album_name": "Close To You"}
]

# Function to insert data into all three tables
def insert_all_data(session, data):
    for row in data:
        # Insert into music_library
        execute_insert(session, query_music, (row['year'], row['artist_name'], row['album_name']))

        # Insert into artist_library
        execute_insert(session, query_artist, (row['artist_name'], row['year'], row['album_name']))

        # Insert into album_library
        execute_insert(session, query_album, (row['album_name'], row['artist_name'], row['year']))

# Call the function to insert all data
insert_all_data(session, data)

Inserted: (1970, 'The Beatles', 'Let it Be')
Inserted: ('The Beatles', 1970, 'Let it Be')
Inserted: ('Let it Be', 'The Beatles', 1970)
Inserted: (1965, 'The Beatles', 'Rubber Soul')
Inserted: ('The Beatles', 1965, 'Rubber Soul')
Inserted: ('Rubber Soul', 'The Beatles', 1965)
Inserted: (1965, 'The Who', 'My Generation')
Inserted: ('The Who', 1965, 'My Generation')
Inserted: ('My Generation', 'The Who', 1965)
Inserted: (1966, 'The Monkees', 'The Monkees')
Inserted: ('The Monkees', 1966, 'The Monkees')
Inserted: ('The Monkees', 'The Monkees', 1966)
Inserted: (1970, 'The Carpenters', 'Close To You')
Inserted: ('The Carpenters', 1970, 'Close To You')
Inserted: ('Close To You', 'The Carpenters', 1970)


In [45]:
query = "select * from music_library WHERE YEAR=1970"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.year, row.artist_name, row.album_name,)

1970 The Beatles Let it Be
1970 The Carpenters Close To You


In [46]:
session.shutdown()
cluster.shutdown()